In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import pandas as pd

from strain_relief.cmdline import strain_relief as sr

/opt/homebrew/Caskroom/miniforge/base/envs/strain/lib/python3.11/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indi

In [2]:
! pip install py3Dmol
import py3Dmol

To run the StrainReleif tool you will need your docked poses in a parquet file. The parquet must contain two columns: your molecule's poses as RDKit.Mol binary strings and a unique ID for each pose.

You likely don't have your poses in this format to begin with so below is a handy function that converts sdfs to this format.

In [3]:
def sdf_to_parquet(sdf_file, parquet_file):
    suppl = Chem.SDMolSupplier(sdf_file, sanitize=False, removeHs=False)
    mols = [mol for mol in suppl if mol is not None]
    
    df = pd.DataFrame([{"mol_bytes": mol.ToBinary(), **mol.GetPropsAsDict()} for mol in mols])
    df = df.reset_index(drop=False, names='id')
    df.to_parquet(parquet_file)

sdf_path = '../data/example_ligboundconf.sdf'
parquet_path = '../data/example_ligboundconf_input.parquet'

sdf_to_parquet(sdf_path, parquet_path)

Now that we have our poses in the correct format, lets create a hydra config for StrainRelief.

In [4]:
with initialize(version_base="1.1", config_path="../src/strain_relief/hydra_config"):
    cfg = compose(
        config_name="default", 
        overrides=[
            "experiment=mmff94s",
            f"io.input.parquet_path={parquet_path}"
            ]
        )

print(OmegaConf.to_yaml(cfg))

seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ../data/example_ligboundconf_input.parquet
    mol_col_name: null
    id_col_name: null
  output:
    output_file: null
    id_col_name: ${..input.id_col_name}
    mol_col_name: ${..input.mol_col_name}
conformers:
  randomSeed: ${..seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${..numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.5
  fexit: 250
global_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.05
  fexit: 250
energy_eval:
  method: ${..global_min.method}



Now we can run the tool! (This should take less than a minute)

(There are more example scripts in `StrainRelief/examples/`. These can be executed from the command line using the `strain-relief` command or replicate them in the cell above with hydra.)

In [5]:
results = sr(cfg)
results.head()

,id,mol_bytes,ligand_id,some_property,charge,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter,nconfs_converged
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3Q4_3QD0_A_370,A,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-122.361323,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-132.501279,10.139956,True,20
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,YTW_6IBK_A_525,B,0,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-14.682596,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-45.220088,30.537492,False,21


The `results` dataframe contains all input columns (in this case `id`, `ligand_id`, `some_property` and `mol_bytes`) and all calculated columns:
- `charge`: RDKit's formal charge. StrainRelief only calculates strains of neutral molecules
- `local_min_mol`: the coordinates of the local minimum
- `local_min_e`: the energy of the local minimum (in kcal/mol)
- `global_min_mol`: the coordinates of the global minimum
- `global_min_e`: the energy of the global minimum (in kcal/mol)
- `ligand_strain`: difference between local and global minima
- `passes_strain_filter`: whether `ligand_strain` is lower than the config threshold
- `nconfs_converged`: the number of conformers that convereged when searching for the global minimum 

Lets have a look at the three poses from ligand 3Q4_3QD0_A_370. 

In [6]:
docked = Chem.Mol(results.mol_bytes[0])
local_min = Chem.Mol(results.local_min_mol[0])
global_min = Chem.Mol(results.global_min_mol[0])

In [7]:
rdDetermineBonds.DetermineBonds(docked)
rdDetermineBonds.DetermineBonds(local_min)
rdDetermineBonds.DetermineBonds(global_min)

In [8]:
IPythonConsole.drawMol3D(docked)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
IPythonConsole.drawMol3D(local_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
IPythonConsole.drawMol3D(global_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The original and local minimum conformers look very similar to the eye. This is because local minimisation has a loose convergence criteria and is simply to clean up any high energy artifacts left my docking. The global minimum is noticably different, with all aromatic rings having relaxed into similar planes.

You may again want to convert your results back into an sdf. You can do this with the function below:

In [11]:
def set_properties(mol, properties, row, pose):
    mol.SetProp("pose", pose)
    for p in properties:
        mol.SetProp(p, str(row[p]))
    return mol

def dataframe_to_sdf(df, sdf_file):
    properties = [p for p in df.columns if p not in {"mol_bytes", "local_min_mol", "global_min_mol"}]
    sdf = Chem.SDWriter(sdf_file)

    for _, row in df.iterrows():
        mol = set_properties(Chem.Mol(row.mol_bytes), properties, row, "docked")
        sdf.write(mol)

        local_min = set_properties(Chem.Mol(row.local_min_mol), properties, row, "local_min")
        sdf.write(local_min)

        global_min = set_properties(Chem.Mol(row.global_min_mol), properties, row, "global_min")
        sdf.write(global_min)
    
    sdf.close()

dataframe_to_sdf(results, '../data/example_ligboundconf_output.sdf')

Hopefully you now have a good grasp on how to run the StrainRelief tool! I hope you find it as useful as we have.

Ewan